In [ ]:
import json
import requests
import os
from dotenv import load_dotenv

load_dotenv()
# API_KEY = os.getenv("CONGRESS_API_KEY")
API_KEY = "cW09gbNMaffaTf3nQczNUKl7XZt5lgHYL40uDDMn" # cadens

BASE_URL = "https://api.congress.gov/v3"
endpoint = "/bill"
congresses = [i for i in range(106, 120)] # year 2000-2025
congresses.reverse()
bill_types = ["hr", "s", "hjres", "sjres"] # house of reps, senate, house joint res, senate joint res
params = {"api_key" : API_KEY,
          "limit" : 1} # 250 is max

COMPLETED = []

congresses

In [ ]:
import xml.etree.ElementTree as ET

for c in congresses:
    if c in COMPLETED:
        print(f"{c} already done")
        continue
    b_data = {}
    for b_t in bill_types:
        b_file_name = f"scraped_json/{c}_{b_t}.json"
        a_file_name = f"scraped_json/{c}_{b_t}_actions.json"
        if os.path.exists(b_file_name) and os.path.exists(a_file_name):
            print(f"{b_file_name} exists")
            print(f"{a_file_name} exists")
            with open(b_file_name, "r") as b_file:
                b_data = json.load(b_file) # bill
                with open(a_file_name, "r") as a_file:
                    a_data = json.load(a_file) # action data
                    for b_bill, a_bill in zip(b_data, a_data):
                        if "actions" not in b_data[b_bill]:
                            print(f"ERROR: {b_file_name}\n{b_bill}")
                            continue
                        if "actions" not in a_data[a_bill]:
                            print(f"ERROR: {a_file_name}\n{a_bill}")
                            continue
                        b_data[b_bill]["actions"] = a_data[a_bill]["actions"]

                for bill in b_data:
                    # if bill != "119_hr_33":
                    #     continue
                    if "actions" not in b_data[bill]:
                        print(f"ERROR: {file_name}\n{bill}")
                        continue
                    actions = b_data[bill]["actions"]
                    if "error" not in actions: # actions have been taken
                        # print(bill)
                        if "actions" in actions:
                            for i in range(len(actions["actions"])):
                                a = actions["actions"][i]
                                if "recordedVotes" in a:
                                    # print(a)
                                    for j in range(len(a["recordedVotes"])):
                                        v = a["recordedVotes"][j]
                                        
                                        response = requests.get(v["url"])
                                        root = ET.fromstring(response.content) 

                                        votes_dict = {"Yea": {}, "Nay": {}}
                                        
                                        if "chamber" in v and v["chamber"] == "Senate":
                                            votes_dict = {"Yea": {}, "Nay": {}}
                                            for member in root.findall(".//member"):
                                                vote_choice = member.find("vote_cast").text
                                                if vote_choice in votes_dict:
                                                    legislator_data = {
                                                        "name": member.find("member_full").text,
                                                        "party": member.find("party").text,
                                                        "state": member.find("state").text,
                                                    }
                                                    legislator_key = f"{legislator_data['name']}_{legislator_data['state']}"
                                                    votes_dict[vote_choice][legislator_key] = legislator_data
                                            # print(votes_dict)
                                        else:
                                            for vote in root.findall(".//recorded-vote"):
                                                legislator = vote.find("legislator")
                                                vote_choice = vote.find("vote").text
                                                if vote_choice in votes_dict:
                                                    space_split = legislator.text.split()  # split if state in name i.e. Scott (FL)
                                                    if len(space_split) > 1 and space_split[-1][0] == "(" and  space_split[-1][-1] == ")":
                                                        legislator.text = ""
                                                        for w in space_split[:-1]:
                                                            legislator.text += w + " "
                                                        legislator.text = legislator.text[:-1] # terminal space
                                                    comma_split = legislator.text.split(",") # split by comma i.e Green,Al
                                                    if len(comma_split) > 1: # space and parenthesis present
                                                        legislator.text = "".join(comma_split[:-1])
                                                    legislator_key = f"{legislator.text}_{legislator.attrib.get("state")}"
                                                    legislator_data = {
                                                        "name": legislator.text,
                                                        "party": legislator.attrib.get("party"),
                                                        "state": legislator.attrib.get("state"),
                                                    }
                                                    votes_dict[vote_choice][legislator_key] = legislator_data
                                        b_data[bill]["actions"]["actions"][i]["recordedVotes"][j]["voteTable"] = votes_dict
                                        # print("added")

            with open(f"compiled_json/{c}_{b_t}_compiled.json", "w") as json_file:
                json.dump(b_data, json_file)